In [ ]:
import os
import openai

from dotenv import load_dotenv
load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")

In [17]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "Hi my name is Earl, how are you?"}
    ],
    functions=[

        {
            "name": "translateToSpanish",
            "description": "Translate to spanish",
            "parameters": {
                'title': 'StepByStepAIResponse',
                'type': 'object',
                'properties': {'english': {'title': 'english', 'type': 'string'},
                               'spanish': {'title': 'spanish', 'type': 'string'}},
                'required': ['english', 'spanish']
            }
        }
    ],
    function_call={"name": "translateToSpanish"}
)

In [18]:
print(res.choices[0]["message"]["function_call"]["arguments"])

{
  "english": "Hi my name is Earl, how are you?",
  "spanish": "Hola, mi nombre es Earl, ¿Cómo estás?"
}


first step is to get intent or http verb
extract the target
query the database for target
extract params
build the api call
confirmwith the user
call the api
display api call status

Here I can extract the HTTP VERB from the context, might need to adjust to work well with UKG's APIs

In [19]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"Only respond in HTTP Verbs such as GET, POST, PUT and DELETE"},
        {"role": "user", "content": "Who is John Snow?"}
    ],
    functions=[

        {
            "name": "promptToHTTPVerb",
            "description": "Extract the http verb from the string and return only a single http verb",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'verb': {'title': 'verb', 'type': 'string'}},
                'required': ['verb']
            }
        }
    ],
    function_call={"name": "promptToHTTPVerb"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])


{
  "verb": "GET"
}


Here I can extract the subject of the users reply

In [20]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"Only respond with the name of the subject of the message."},
        {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
    ],
    functions=[
        {
            "name": "extractSubject",
            "description": "Extract the subject of a users reply",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'subject': {'title': 'subject', 'type': 'string'}},
                'required': ['subject']
            }
        }
    ],
    function_call={"name": "extractSubject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])

{
  "subject": "John Snow"
}


Here I try to generate the params fo send a get request to the UKG API

In [16]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "assistant", "content": ''' Generate a headers and params object to make a call to this endpoint https://api.ukg.com/employee
            {
            "headers" = {
                            "Authorization": "Bearer api_key",
                            "Content-Type": "application/json"
                        },
            "params" = {
                            "firstName": first_name,
                            "lastName": last_name
                        }
            }
'''},
        {"role": "user", "content": "Update John Snow to Jogn Ramsey"}
    ],
    functions=[

        {
            "name": "extractSubject",
            "description": "Extract the subject of a users reply",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'object': {'object': {
            "headers"={
                            "Authorization": "Bearer api_key",
                            "Content-Type": "application/json"
                        },
            "params"={
                            "firstName": "first_name",
                            "lastName": "last_name"
                        }
            },
    "params"={
        "firstName": first_name,
        "lastName": last_name
    }}, 'type': 'object'}},
                'required': ['object']
            }
        }
    ],
    function_call = {"name": "extractSubject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])


{
"subject": "John Snow"
}


In [ ]:
import requests

def get_employee_data(first_name, last_name):
    base_url = "https://api.ukg.com/employee"
    api_key = "YOUR_API_KEY"  # Replace with your actual API key

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    params = {
        "firstName": first_name,
        "lastName": last_name
    }

    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
employee_data = get_employee_data("John", "Snow")

if employee_data:
    # Process the employee data
    print("Employee Data:")
    print(f"Name: {employee_data['name']}")
    print(f"Department: {employee_data['department']}")
    # Add more fields as needed
else:
    print("No employee data found for the given name.")
